In [51]:
import jax
import symjax as sj
import symjax.tensor as T
import numpy as np

In [52]:
def updatetuple(tup, idx, val):
    inplist = list(tup)
    inplist[idx] = val
    return tuple(inplist)

In [71]:
def body_func(inp):
    print("in body function")
    tiprime_iter = inp[0]
    ti_iter = inp[1]
    lambda_list = inp[4]
    phi_list = inp[5]

    # test = T.index_update(inp[4], (tiprime_iter,ti_iter), 2)
    # test = inp[4].at[(0,1)].set(2)
    # inp = updatetuple(inp, 4, test)

    # test = T.index_update(inp, 4, T.append(inp[4], 1))
    newl = T.append(inp[4], 1)
    newp = T.append(inp[5], 1)

    # carry[ti_iter, tiprime_iter] = ti_iter 
    return (tiprime_iter + 1, ti_iter + 1, inp[2], inp[3], newl, newp)

In [72]:
def cond_funct(inp):
    print("in cond function")
    tip_iter = inp[0]
    ti_iter = inp[1]
    dim1max = inp[2]
    dim2max = inp[3]
    # print(dim1max)
    # print(tip_iter)
    # return T.less_equal(tip_iter, dim1max)
    return T.logical_and(T.less_equal(tip_iter, dim1max), T.less_equal(ti_iter, dim2max))

In [73]:
signal = T.Placeholder((2,), "float32", name="signal")
dim1ph = T.Variable(2, "float32", "alpha")
dim2ph = T.Variable(2, "float32", "alpha")
lph = T.Placeholder((1,), "float32", name = "lambda")
pph = T.Placeholder((1,), "float32", name = "phi")
# ct = -np.ones(np.min([2,2]))
# nph = T.Variable(n, "float32", "alpha")
output = T.while_loop(
    lambda a: cond_funct(a), # condition
    lambda a: body_func(a), # body
    [signal[0], signal[1], dim1ph, dim2ph, lph, pph], # init 
)
f = sj.function(signal, lph, pph, outputs=output)

in cond function
in body function


TypeError: body_fun output and input must have identical types, got
(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(int32[]), ShapedArray(int32[]), ShapedArray(float32[2]), ShapedArray(float32[2]))
and
(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(int32[]), ShapedArray(int32[]), ShapedArray(float32[1]), ShapedArray(float32[1])).

In [64]:
l = np.array([0])
p = np.array([0])
final_state = f(np.array([0,0]), l, p)

in cond function
in body function


In [65]:
final_state

(array(3, dtype=int32),
 array(3, dtype=int32),
 array(2, dtype=int32),
 array(2, dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32))

In [66]:
l

array([0])

In [67]:
p

array([0])